In [1]:
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
from tensorflow.keras.models import load_model
vggface_model = load_model("/kaggle/input/resnet50/resnet50face.h5")

In [ ]:
train = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/sample_submission.csv')
test.head()

In [ ]:
print("Кол-во классов: {}".format(train['emotion'].nunique()))

In [2]:
from keras import backend as K

def preprocess_input(x, data_format=None, version=1):
    x_temp = np.copy(x)
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if version == 1:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

    elif version == 2:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
    else:
        raise NotImplementedError

    return x_temp

In [3]:
def preprocess_input_facenet(image_):

    preprocessed = preprocess_input(image_, version=2)
    
    return preprocessed

image_gen = ImageDataGenerator(preprocessing_function=preprocess_input_facenet) 

In [4]:
BATCH_SIZE = 128
IMAGE_SIZE = 224

train_generator = image_gen.flow_from_directory(directory=str('/kaggle/input/train-data-plus-plus'),
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                class_mode='categorical')

Found 150213 images belonging to 1 classes.


In [ ]:
base_model = tf.keras.Model([vggface_model.input], vggface_model.get_layer("activation_49").output)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization 

model = tf.keras.Sequential([
  base_model,
  Dropout(0.5),
  Conv2D(2048, (3, 3),activation='relu',padding='same'),
  Conv2D(2048, (3, 3),activation='relu',padding='same'),
  BatchNormalization(),
  MaxPooling2D(pool_size=(7, 7)),
  Dropout(0.5),
  Flatten(), # Конвертация в двумерный тензор
  Dense(256, activation='relu'),
  Dense(128, activation='relu'),
  BatchNormalization(),
  Dropout(0.2),
  Dense(9, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="categorical_crossentropy",
              metrics=['accuracy'])


In [ ]:
%%timeit -n 10 -r 10

test = model(np.random.rand(1,224,224,3).astype(np.float32))

In [ ]:
history = model.fit(train_generator, epochs=1, batch_size=BATCH_SIZE)

In [ ]:
predict_dict = {0:'anger',1:'contempt',2:'disgust',3:'fear',4:'happy',5:'neutral',6:'sad',7:'surprise',8:'uncertain'}

In [ ]:
test_data = image_gen.flow_from_directory(
        '/kaggle/input/test-data',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE)

In [ ]:
emotion = model.predict(test_data)

In [ ]:
emotion_cm = [predict_dict[np.argmax(i)] for i in emotion]

In [ ]:
test['emotion'] = emotion_cm
test.head()

In [ ]:
test.to_csv('/kaggle/working/submission.csv', sep=',', index=False)

В этом ноутбуке я решил использовать метод transfer learning и finetuning. Я взял за основу сеть, которая была обучена на датасете VGGFace2 для классификации лиц. Только в этот раз я заморозил слой модели до "activation_49" и дополнил дургими слоями. Здесь осталась ошибка с неправильным датасетом тестовых данных. Score на обучающих данных: 0.4179